# Very simple agent

This notebook show how to create a very simple conversation agent, with no tools.

### Let's start by importing the necessary libraries and set the api keys

In [1]:
from dotenv import load_dotenv

from typing import List
from agente.core.base import BaseAgent

## Load and set environment variables from .env file
load_dotenv()

True

### This first example show how to create an agent without streaming.

We note that even the agent is set to run without stream, we need to iterate over agent.run() to get the messages. 
This allows to stream in case a second agent with stream is called by the first parent agent.

In [2]:
class VerySimpleAgent(BaseAgent):
    agent_name: str = "VerySimpleAgent"
    system_prompt:str = "You are a helpful AI assistant."
    completion_kwargs: dict = {
        "model": "gpt-4o",
        "stream": False,
        "temperature": 1.0,
        "max_tokens": 500,
    }
    
agent = VerySimpleAgent()
agent.add_message("user", "Please write me a funny joke.")
responses = [r async for r in agent.run()] 
responses[-1].dict()

{'call_id': 'chatcmpl-AvBRrSewbK2CKEqnYzm540V2OF5a6',
 'agent_name': 'VerySimpleAgent',
 'role': 'assistant',
 'content': "Why don't scientists trust atoms?\n\nBecause they make up everything!",
 'tool_calls': [],
 'usage': {'completion_tokens': 13, 'prompt_tokens': 25, 'total_tokens': 38}}

### The conversation history is stored as well:

In [3]:
agent.conv_history.messages

[Message(role='system', agent_name='VerySimpleAgent', content='You are a helpful AI assistant.', tool_calls=None, tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None),
 Message(role='user', agent_name='VerySimpleAgent', content='Please write me a funny joke.', tool_calls=None, tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None),
 Message(role='assistant', agent_name='VerySimpleAgent', content="Why don't scientists trust atoms?\n\nBecause they make up everything!", tool_calls=None, tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None)]

## Streaming

### Now let's create the same agent but with stream. For that we can use Gradio for better visualization

In [4]:
import gradio as gr

class VerySimpleAgent(BaseAgent):
    agent_name: str = "VerySimpleAgent"
    completion_kwargs: dict = {
        "model": "gpt-4o",
        "stream": True,
        "temperature": 1.0,
        "max_tokens": 500,
    }


def get_new_agent():
    """Create a fresh agent instance"""
    new_agent = VerySimpleAgent()
    return new_agent

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    # Initialize with a function call instead of direct instantiation
    main_agent = gr.State(value=None)

    def user(user_message, agent):
        if agent is None:
            agent = get_new_agent()
        agent.add_message("user", user_message)
        return "", [{"role": "user", "content": user_message}], agent

    async def bot(history, agent):
        if agent is None:
            agent = get_new_agent()
        if not history:
            yield [], agent
            return
        
        history.append({"role": "assistant", "content": ""})
        async for chunk in agent.run():
            if chunk.content:
                history[-1]["content"] += chunk.content
                yield history, agent

    def reset_state():
        return None, get_new_agent()

    msg.submit(user, [msg, main_agent], [msg, chatbot, main_agent]).then(
        bot, [chatbot, main_agent], [chatbot, main_agent]
    )
    clear.click(reset_state, None, [chatbot, main_agent])

demo.launch()

/Users/miguelwon/miniconda3/envs/agente/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
